<a href="https://colab.research.google.com/github/suhailnajeeb/deep-learning-chronicles/blob/main/2012_Alexnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ImageNet Classification with Deep Convolutional Neural Networks (2012)**  

[Link to paper](https://proceedings.neurips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf)  

Summary: 

* *Architecture:* 5-Layer Conv, 3-Fully Connected
* *Activation:* ReLU
* *Local Response Normalization:* Aids Generalization
* *Overlapping Pooling:* Reduces Overfitting
* *Data Augmentation:* 
    * Create patches, Train on Reflections
    * Multiple Classifiers - Combine Results
    * Altering intensities of RGB Channels  
    (Add Multiples of Principal Components)
* *Droupout:* Allows to simulate the training of multiple models and Drastically reduces Overfitting
* *Learning:* 
    * *Optimizer:* Stochastic Gradient Descent
    * *Momentum:* 0.9
    * *Batch Size:* 128
    * *Weight Decay:* 0.0005
    * *Initialization:* Zero-mean Gaussian Distribution,  
    Biases of 2nd, 4th, 5th Conv Layers set to 1 for ReLU

The architecture is forked from the following Github Repository:   
https://github.com/dansuh17/alexnet-pytorch

In [2]:
import torch
from torch import nn

In [ ]:
# Reference: https://github.com/dansuh17/alexnet-pytorch

class AlexNet(nn.Module):
    def __init__(self, num_classes = 1000):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4),
            nn.ReLU(),
            nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(96, 256, 5, padding = 2),
            nn.ReLU(),
            nn.LocalResponseNorm(size = 5, alpha = 0.0001, beta = 0.75, k = 2),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
            nn.Conv2d(256, 384, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5, inplace = True),
            nn.Linear(in_features = (256*6*6), out_features = 4096),
            nn.ReLU(),
            nn.Dropout(p = 0.5, inplace = True),
            nn.Linear(in_features = 4096, out_features = 4096),
            nn.ReLU(),
            nn.Linear(in_features = 4096, out_features = num_classes),
        )
        self.init_bias()
    
    def init_bias(self):
        for layer in self.net:
            if isinstance(layer, nn.Conv2d):
                nn.init.normal_(layer.weight, mean = 0, std = 0.01)
                nn.init.constant_(layer.bias, 0)
        nn.init.constant_(self.net[4].bias, 1)
        nn.init.constant_(self.net[10].bias, 1)
        nn.init.constant_(self.net[12].bias, 1)

    def forward(self, x):
        x = self.net(x)
        x = x.view(-1, 256*6*6)
        return self.classifier(x)

In [ ]:
model = AlexNet()

In [ ]:
x = torch.zeros(1, 3, 227, 227)

In [ ]:
out = model(x)

In [ ]:
out.shape

torch.Size([1, 1000])

The following implementation is from PyTorch source-code:

https://pytorch.org/docs/stable/_modules/torchvision/models/alexnet.html

In [3]:
# This implementation does not use the Local response Norm
# And uses Default Initialization instead of the paper-specific one.

class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

It can be directly imported from torchvision: 

In [4]:
from torchvision.models import alexnet

model = alexnet()

In [5]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 